In [1]:
import funzioni02 as f2
import funzioni01 as f1
import qiskit.quantum_info as qi
from copy import deepcopy
from sympy import *
import numpy as np
import qiskit.ignis.mitigation.measurement as mc
from qiskit import Aer, assemble, QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, transpile, execute
from qiskit.providers.aer import AerSimulator, QasmSimulator
from qiskit.opflow import Zero, One, I, X, Y, Z
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity, Statevector
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
plt.rcParams.update({'font.size': 16})  # enlarge matplotlib fonts
plt.rcParams['figure.figsize'] = [15, 10]
%load_ext autoreload
%autoreload 2

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')

sim_noisy_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))
backend_sim_jakarta = sim_noisy_jakarta 
backend = provider.get_backend('ibmq_jakarta')
backend_sim = Aer.get_backend('qasm_simulator')

In [2]:
shots=32000
precision=50
steps=42
backend_aus=backend_sim_jakarta
check=["4copy_check", [0,2,4,6]]
evo_type="complete_evolution" # "column_evolution" or "complete_evolution" or "trotter_steps"
time=np.pi/2

In [3]:
qr_cal = QuantumRegister(7)
qcs, qcs_na = f2.evolution_tomo(type=evo_type, N_steps=steps, tempo=time, precision=precision, initial_state='110', check=check)
cal_circ, state_labels = mc.complete_meas_cal(qubit_list=[1,3,5,0,2,4,6], qr=qr_cal, circlabel='mcal') #[1,3,5,0,2,4]


In [4]:
qcs_na[-1].draw()

┌──────────────┐ ░          
 q_0: ──────────────────────┤0             ├─░──────────
           ┌────┐┌─────────┐│              │ ░ ┌─┐      
 q_1: ─────┤0   ├┤0        ├┤4             ├─░─┤M├──────
           │    ││         ││              │ ░ └╥┘      
 q_2: ─────┤  U ├┤         ├┤1             ├─░──╫───────
      ┌───┐│    ││         ││              │ ░  ║ ┌─┐   
 q_3: ┤ X ├┤1   ├┤1 Mdg_dg ├┤5 4copy_check ├─░──╫─┤M├───
      └───┘└────┘│         ││              │ ░  ║ └╥┘   
 q_4: ───────────┤         ├┤2             ├─░──╫──╫────
      ┌───┐      │         ││              │ ░  ║  ║ ┌─┐
 q_5: ┤ X ├──────┤2        ├┤6             ├─░──╫──╫─┤M├
      └───┘      └─────────┘│              │ ░  ║  ║ └╥┘
 q_6: ──────────────────────┤3             ├─░──╫──╫──╫─
                            └──────────────┘ ░  ║  ║  ║ 
c0: 3/══════════════════════════════════════════╩══╩══╩═
                                                0  1  2

In [5]:
qcs_calib, _ = f2.calibration_cirquits(type="itself_whole", n_steps=steps, time=time, q_anc=check[1], check="yes", check_type=check[0])


In [6]:
#qc = qcs_calib[0].copy()
#qc.remove_final_measurements()
#U = f2.matrix_from_cirquit(qc, type="numpy")
#Matrix(U)

In [7]:
job=execute(qcs, backend_aus, shots=shots)

job_cal_our=execute(qcs_calib, backend = backend_aus, shots=shots)

job_cal=execute(cal_circ, backend_aus, shots=shots)

In [8]:
qc_ausiliare = QuantumCircuit(7)

qc_ausiliare.swap(5,4)
qc_ausiliare.swap(5,2)
qc_ausiliare.swap(1,3)
qc_ausiliare.swap(0,1)

qc_ausiliare2=qcs_calib[0].copy()
qc_ausiliare2.remove_final_measurements()

qc_ausiliare.append(qc_ausiliare2,[0,1,2,3,4,5,6])

qc_ausiliare.swap(0,1)
qc_ausiliare.swap(1,3)
qc_ausiliare.swap(5,2)
qc_ausiliare.swap(5,4)

U = f2.matrix_from_cirquit(qc_ausiliare, type="numpy")
# Matrix(U)

In [9]:
meas_fitter_our = mc.CompleteMeasFitter(job_cal_our.result(), state_labels=state_labels)
meas_fitter = mc.CompleteMeasFitter(job_cal.result(), state_labels=state_labels)

U_tilde=meas_fitter_our.cal_matrix

In [10]:
qr_basi = QuantumRegister(7)
qc_basi = QuantumCircuit(qr_basi)

#qc_basi.swap(5,4)
#qc_basi.swap(5,2)
#qc_basi.swap(1,3)
#qc_basi.swap(0,1)

qcs_basi = state_tomography_circuits(qc_basi,[0,1,2])


C_matrices = []
C=U_tilde @ np.asarray(np.matrix(U).getH())

for qca in qcs_basi:
    
    qca.remove_final_measurements()

    #qca.swap(0,1)
    #qca.swap(1,3)
    #qca.swap(5,2)
    #qca.swap(5,4)

    qca_matrix = np.matrix(f2.matrix_from_cirquit(qca, type="numpy"))
    qca_matrixH = np.asarray(qca_matrix.getH())

    C_aus = qca_matrix @ (C @ qca_matrixH)

    C_matrices.append(np.asarray(C_aus))

In [11]:
from copy import deepcopy
meas_fitter_ours = []

for C_new in C_matrices:
    meas_fitter_our_aus = deepcopy(meas_fitter_our)
    meas_fitter_our_aus._tens_fitt.cal_matrices[0]=C_new

    meas_fitter_ours.append(meas_fitter_our_aus)



In [12]:
np.shape(meas_fitter.cal_matrix)

(128, 128)

In [13]:
np.shape(meas_fitter_ours[0].cal_matrix)

(128, 128)

In [14]:
U = f2.Trotter_N_approx(steps=steps, tempo=time, precision=precision)

U2=[
    [U[3*8+3],U[3*8+5],U[3*8+6],0],
    [U[5*8+3],U[5*8+5],U[5*8+6],0],
    [U[6*8+3],U[6*8+5],U[6*8+6],0],
    [0,0,0,1]
]

qc=QuantumCircuit(2, name="U, t=%.2f pi"%(time/np.pi))
qc.unitary(U2,[0,1])    
trans_qc=transpile(qc,basis_gates=['cx','x','sx','rz']) 

qr2=QuantumRegister(3, name="q")
qc2=QuantumCircuit(qr2)

qc2.x([qr2[1],qr2[2]])
qc2.append(trans_qc,[qr2[0],qr2[1]])
qc2.append(f2.Mdg_circquit().inverse(),[qr2[0],qr2[1],qr2[2]])

target_state = Statevector.from_instruction(qc2)

In [16]:
#qcs2=
qc2.draw()

┌───────────────┐┌─────────┐
q_0: ─────┤0              ├┤0        ├
     ┌───┐│  U, t=0.50 pi ││         │
q_1: ┤ X ├┤1              ├┤1 Mdg_dg ├
     ├───┤└───────────────┘│         │
q_2: ┤ X ├─────────────────┤2        ├
     └───┘                 └─────────┘

In [17]:
target_state = (np.asarray(target_state))
target_state

array([ 2.20970511e-30+2.13057032e-30j, -2.01216369e-18-2.55718904e-18j,
        9.76774938e-18-1.36065015e-18j,  6.66748044e-01+1.46220568e-03j,
        2.23705418e-17-7.60132515e-19j,  6.66583679e-01-7.31102841e-04j,
       -3.33331723e-01-7.31102841e-04j,  1.46459212e-14+1.09755027e-14j])

In [18]:
target_state=((Zero^One^One).to_matrix())


In [19]:
reps=1

#target_state = (One^One^Zero).to_matrix()

fids=np.zeros([reps,4])
fids_mean=np.zeros(4)
fids_dev=np.zeros(4)

for j in range(reps):
    
    #res = jobs_evo_result[j]
    res=job.result()

    print(j)
    
    new_res, new_res_nm = f2.mitigate(res, ancillas_conditions=['0011','1110','1101'], meas_fitter=meas_fitter)
    new_res_our, new_res_nm = f2.mitigate(res, ancillas_conditions=['0011','1110','1101'], meas_fitter=meas_fitter_ours)
    new_res_not_mitigated = f2.mitigate(res, ancillas_conditions=f2.bin_list(4))

    fids[j,0] = f2.fidelity_count(new_res_not_mitigated, qcs_na, target_state)
    fids[j,1] = f2.fidelity_count(new_res_nm, qcs_na, target_state)
    fids[j,2] = f2.fidelity_count(new_res, qcs_na, target_state)
    fids[j,3] = f2.fidelity_count(new_res_our, qcs_na, target_state)

for i in range(4):
    fids_mean[i]=np.mean(fids[:,i])
    fids_dev[i]=np.std(fids[:,i])


0
0
tens_fitt
1
tens_fitt
2
tens_fitt
3
tens_fitt
4
tens_fitt
5
tens_fitt
6
tens_fitt
7
tens_fitt
8
tens_fitt
9
tens_fitt
10
tens_fitt
11
tens_fitt
12
tens_fitt
13
tens_fitt
14
tens_fitt
15
tens_fitt
16
tens_fitt
17
tens_fitt
18
tens_fitt
19
tens_fitt
20
tens_fitt
21
tens_fitt
22
tens_fitt
23
tens_fitt
24
tens_fitt
25
tens_fitt
26
tens_fitt
0
0 (128, 128)
deepcopy
1
1 (128, 128)
deepcopy
2
2 (128, 128)
deepcopy
3
3 (128, 128)
deepcopy
4
4 (128, 128)
deepcopy
5
5 (128, 128)
deepcopy
6
6 (128, 128)
deepcopy
7
7 (128, 128)
deepcopy
8
8 (128, 128)
deepcopy
9
9 (128, 128)
deepcopy
10
10 (128, 128)
deepcopy
11
11 (128, 128)
deepcopy
12
12 (128, 128)
deepcopy
13
13 (128, 128)
deepcopy
14
14 (128, 128)
deepcopy
15
15 (128, 128)
deepcopy
16
16 (128, 128)
deepcopy
17
17 (128, 128)
deepcopy
18
18 (128, 128)
deepcopy
19
19 (128, 128)
deepcopy
20
20 (128, 128)
deepcopy
21
21 (128, 128)
deepcopy
22
22 (128, 128)
deepcopy
23
23 (128, 128)
deepcopy
24
24 (128, 128)
deepcopy
25
25 (128, 128)
deepcopy
2

In [20]:
fids_mean

array([0.33061967, 0.37351316, 0.41233388, 0.47023094])

In [26]:
initial_state = One^One^Zero
target_state2 = Statevector.from_instruction(qc2)#Zero^One^One
from qiskit.opflow import *

print(type(initial_state), type(target_state2))#, type(MatrixOp(np.asarray(target_state2))))

np.abs((~target_state2 @ U_heis3(float(time)) @ initial_state).eval())**2

<class 'qiskit.opflow.state_fns.dict_state_fn.DictStateFn'> <class 'qiskit.quantum_info.states.statevector.Statevector'>


TypeError: bad operand type for unary ~: 'Statevector'

In [ ]:
def H_heis3():
    # Interactions (I is the identity matrix; X, Y, and Z are Pauli matricies; ^ is a tensor product)
    XXs = (I^X^X) + (X^X^I)
    YYs = (I^Y^Y) + (Y^Y^I)
    ZZs = (I^Z^Z) + (Z^Z^I)
    
    # Sum interactions
    H = XXs + YYs + ZZs
    
    # Return Hamiltonian
    return H

def U_heis3(t):
    # Compute XXX Hamiltonian for 3 spins in a line
    H = H_heis3()
    
    # Return the exponential of -i multipled by time t multipled by the 3 spin XXX Heisenberg Hamilonian 
    return (t * H).exp_i()

$\hbar$